In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as ipd

import scipy.io as sio
import librosa

import time

import skimage
import skimage.io
from skimage.transform import rescale

from scipy.io import wavfile
import os
import subprocess

import sys
sys.path.append("../src")
from spectrogramtools import *
from stego import *
from swpowerstego import *
from imtools import *
from tsp import *
from curvature import *

In [ ]:
# Image parameters
thresh = 0.1
p = 1
canny_sigma = 3
density_fac= 1
edge_weight = 10
target_points = 2000

np.random.seed(0)
I = skimage.io.imread("../data/images/usher.jpg")
X = voronoi_stipple(I, thresh, target_points=target_points, p=p, canny_sigma=canny_sigma, edge_weight=edge_weight, n_iters=10, do_plot=False)
X = get_tsp_tour(X)
s = get_arclen(get_curv_vectors(X, 0, 1, loop=True)[1])
X = arclen_resample(X, s, X.shape[0])
sigma = 1
X = get_curv_vectors(X, 0, sigma, loop=True)[0]

In [ ]:
# Audio parameters
x, sr = librosa.load("../data/tunes/usher-baby.mp3", sr=44100)
win_length = 1024
mag_idxs = [1, 2]
phase_idxs = mag_idxs

# Sliding window parameters
win = 16
fit_lam = 0.1
q = -1

fac = 0.4
plt.figure(figsize=(30*fac, 12*fac))

sp = STFTPowerDisjoint(x, X, win_length, mag_idxs, phase_idxs, win, fit_lam, q, do_viterbi=False)
Y = sp.get_target(normalize=False)
Z = sp.get_signal(normalize=False)
cost1 = np.sqrt(np.sum((Y[:, 0]-Z[:, 0])**2))
cost2 = np.sqrt(np.sum((Y[:, 1]-Z[:, 1])**2))

plt.subplot(221)
plt.plot(Y[:, 0])
plt.plot(Z[:, 0])
plt.legend(["Target", "SW-STFT"])
plt.title("Cost = {:.0f}".format(cost1))
plt.subplot(222)
plt.plot(Y[:, 1])
plt.plot(Z[:, 1])
plt.title("Cost = {:.0f}".format(cost2))
plt.legend(["Target", "SW-STFT"])

sp = STFTPowerDisjoint(x, X, win_length, mag_idxs, phase_idxs, win, fit_lam, q, do_viterbi=True)
Y = sp.get_target(normalize=False)
Z = sp.get_signal(normalize=False)
cost1 = np.sqrt(np.sum((Y[:, 0]-Z[:, 0])**2))
cost2 = np.sqrt(np.sum((Y[:, 1]-Z[:, 1])**2))

plt.subplot(223)
plt.plot(Y[:, 0])
plt.plot(Z[:, 0])
plt.title("Cost = {:.0f}".format(cost1))
plt.legend(["Target", "SW-STFT"])
plt.subplot(224)
plt.plot(Y[:, 1])
plt.plot(Z[:, 1])
plt.title("Cost = {:.0f}".format(cost2))
plt.legend(["Target", "SW-STFT"])
plt.tight_layout()
plt.savefig("Viterbi.svg", bbox_inches='tight')